# Neighborhood Recommendations for Transfer Employees Through K-Means Clustering

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#Section_1"> Define a Function that Makes a Dashboard </a></li>
    <li><a href="#Section_2">Question 1: Create a dataframe that contains the GDP data and display it</a> </li>
    <li><a href="#Section_3">Question 2: Create a dataframe that contains the unemployment data and display it</a></li>
    <li><a href="#Section_4">Question 3: Display a dataframe where unemployment was greater than 8.5%</a></li>
    <li><a href="#Section_5">Question 4: Use the function make_dashboard to make a dashboard</a></li>
     <li><a href="#Section_6">Question 5: Save the dashboard on IBM cloud and display it</a></li>
    </ul>
<p>
    Estimated Time Needed: <strong>180 min</strong></p>
</div>

<hr>

### Week 1: A description of the problem and a discussion of the background.  (15 marks)

### Week 1: A description of the data and how it will be used to solve the problem. (15 marks)

### Week 2: A link to your Notebook on your Github repository, showing your code. (15 marks)

### Week 2: A full report consisting of all of the following components (15 marks):

### Week 2: Introduction where you discuss the business problem and who would be interested in this project.

### Week 2: Data where you describe the data that will be used to solve the problem and the source of the data.

### Week 2: Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

### Week 2: Results section where you discuss the results.

### Week 2: Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

### Week 2: Conclusion section where you conclude the report.

#### Import the necessary python libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# !conda install -c conda-forge wikipedia --yes
import wikipedia as wp # library to read wikipedia pages.

print('Libraries imported.')

Libraries imported.


In [ ]:
!conda install -c conda-forge geopandas --yes

#### Set the width of the notebook to 98% for easy viewing

In [20]:
# Set up this notebook so that it's easy to work with.
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 98%; }
    div#menubar-container     { width: 75%; }
    div#maintoolbar-container { width: 99%; }   
</style>
""")
)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import os
path="D:\Course\Applied.Data.Science.Capstone"
os.chdir(path)

#### Gather the necessary data for your analysis

#### Get London Neighborhoods and their respective latitudes and longitudes

In [ ]:
# Get wikipedia dataset for London
# https://en.wikipedia.org/wiki/List_of_areas_of_London
London_html = wp.page("List_of_areas_of_London").html().encode("UTF-16")

# Get the table containing the 46 city districts of Frankfurt on the Wikipedia page
df_London = pd.read_html(London_html, header = 0)[1][['Location']]

# Rename column name from location to Neighborhood.
df_London.rename(columns={'Location':'Neighborhood'}, inplace=True)

# Create a new column called City and assign london to all rows
df_London = df_London.assign(City = 'London')

# Remove words within parentheses, along with the parentheses
df_London['Neighborhood'] = df_London['Neighborhood'].str.replace(r"\(.*\)","")

# Save the above created dataframe, remove duplicates and import back the cleanup list of neighborhoods.
df_London = pd.read_excel('London_Neighborhoods.xlsx')

# Get the latitude and Longitude of London Neighborhoods
geo_London = Nominatim(user_agent="Lon_Explorer")
df_London['Latitude'] = df_London['Neighborhood'].apply(geo_London.geocode).apply(lambda x: (x.latitude))
df_London['Longitude'] = df_London['Neighborhood'].apply(geo_London.geocode).apply(lambda x: (x.longitude))

# Rearrange columns for ease of use
df_London = df_London[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_London.shape)
df_London.head()

In [63]:
df1 = pd.read_excel('London_Neighborhoods.xlsx')

In [64]:
df1 = df1.assign(Latitude = 0.01)
df1 = df1.assign(Longitude = 0.01)

In [ ]:
for i in df1.index:
    # print(i)
    geolocator = Nominatim(user_agent="Lon_Explorer", country_bias="UK")
    address = df1.at[i, 'Neighborhood']
    location = geolocator.geocode(address)
    df1.at[i, 'Latitude'] = location.latitude
    df1.at[i, 'Longitude'] = location.longitude

In [57]:
df1.head()

,Neighborhood,City,Latitude,Longitude
0,Abbey Wood,London,51,0
1,Acton,London,51,0
2,Addington,London,47,-66
3,Addiscombe,London,51,0
4,Albany Park,London,41,-87


#### Get Frankfurt Neighborhoods and their respective latitudes and longitudes

In [32]:
# Get wikipedia dataset for Frankfurt
# https://en.wikipedia.org/wiki/Frankfurt
Frankfurt_html = wp.page("Frankfurt").html().encode("UTF-16")

# Get the table containing the 46 city districts of Frankfurt on the Wikipedia page
df_Frankfurt = pd.read_html(Frankfurt_html, header = 0)[4][['City district (Stadtteil)']] # Table titled: "Population of the 46 city districts on 31 December 2009"

# Drop the last row, because it has summary information of Frankfurt.
df_Frankfurt.drop(df_Frankfurt.tail(1).index,inplace=True) # drop last n rows

# Rename column name from location to Neighborhood.
df_Frankfurt.rename(columns={'City district (Stadtteil)':'Neighborhood'}, inplace=True)

# Create a new column called City and assign london to all rows
df_Frankfurt = df_Frankfurt.assign(City = 'Frankfurt')

# Get the latitude and Longitude of Frankfurt Neighborhoods
geo_Frank = Nominatim(user_agent="Frankfurt_Explorer")
df_Frankfurt['Latitude'] = df_Frankfurt['Neighborhood'].apply(geo_Frank.geocode).apply(lambda x: (x.latitude))
df_Frankfurt['Longitude'] = df_Frankfurt['Neighborhood'].apply(geo_Frank.geocode).apply(lambda x: (x.longitude))

# Rearrange columns for ease of use
df_Frankfurt = df_Frankfurt[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_Frankfurt.shape)
df_Frankfurt.head()

(46, 4)


,City,Neighborhood,Latitude,Longitude
0,Frankfurt,Altstadt,31.504619,34.464127
1,Frankfurt,Innenstadt,50.112878,8.674922
2,Frankfurt,Bahnhofsviertel,50.107741,8.668736
3,Frankfurt,Westend-Süd,50.115245,8.662270
4,Frankfurt,Westend-Nord,50.126356,8.667921


#### Get New York City Neighborhoods and their respective latitudes and longitudes

In [33]:
# We'll reuse the New York Neighborhoods data that we prepared in Week 3 of the class
df_Newyork = pd.read_excel('NYC_Neighborhoods.xlsx', usecols="B:D")

# Create a new column called City and assign london to all rows
df_Newyork = df_Newyork.assign(City = 'New York')

# Rearrange columns for ease of use
df_Newyork = df_Newyork[['City', 'Neighborhood', 'Latitude', 'Longitude']]

# Print basic info about dataframe
print(df_Newyork.shape)
df_Newyork.head()

(306, 4)


,City,Neighborhood,Latitude,Longitude
0,New York,Wakefield,40.894705,-73.847201
1,New York,Co-op City,40.874294,-73.829939
2,New York,Eastchester,40.887556,-73.827806
3,New York,Fieldston,40.895437,-73.905643
4,New York,Riverdale,40.890834,-73.912585


In [ ]:
# create map of Frankfurt using latitude and longitude values
map_frankfurt = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(df_Frankfurt['Latitude'], df_Frankfurt['Longitude'], df_Frankfurt['City district (Stadtteil)']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_frankfurt)  
    
map_frankfurt

In [ ]:
# @hidden
CLIENT_ID = 'NEQM0YQY35OBBVOBPETJBLMBYQQZZKXW2CNA1AFVQTCXJLO0' # your Foursquare ID
CLIENT_SECRET = '0MQNFC43KVAHSDZQ2JZDRGZ5WIR4YJE4ZGLWEP22UFFXSCH1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## 2. Explore Neighborhoods in Frankfurt

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

frankfurt_venues = getNearbyVenues(names=df_Frankfurt['City district (Stadtteil)'],
                                   latitudes=df_Frankfurt['Latitude'],
                                   longitudes=df_Frankfurt['Longitude']
                                  )
print(frankfurt_venues.shape)
frankfurt_venues.head()

Altstadt
Innenstadt
Bahnhofsviertel
Westend-Süd
Westend-Nord
Nordend-West
Nordend-Ost
Ostend
Bornheim
Gutleutviertel
Gallus
Bockenheim
Sachsenhausen-Nord
Sachsenhausen-Süd
Flughafen
Oberrad
Niederrad
Schwanheim
Griesheim
Rödelheim
Hausen
Praunheim
Heddernheim
Niederursel
Ginnheim
Dornbusch
Eschersheim
Eckenheim
Preungesheim
Bonames
Berkersheim
Riederwald
Seckbach
Fechenheim
Höchst
Nied
Sindlingen
Zeilsheim
Unterliederbach
Sossenheim
Nieder-Erlenbach
Kalbach-Riedberg
Harheim
Nieder-Eschbach
Bergen-Enkheim
Frankfurter Berg
(646, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,31.504619,34.464127,حمام السمرة,31.504831,34.465252,Spa
1,Altstadt,31.504619,34.464127,Alnaffar Bowtique,31.505054,34.462776,Boutique
2,Altstadt,31.504619,34.464127,Downtown dental clinic,31.507576,34.462728,Health & Beauty Service
3,Altstadt,31.504619,34.464127,Mazaya cafe,31.507920,34.466423,Bistro
4,Altstadt,31.504619,34.464127,In The Hell!,31.501540,34.467350,Moving Target


In [48]:
frankfurt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Altstadt,5,5,5,5,5,5
Bahnhofsviertel,100,100,100,100,100,100
Bergen-Enkheim,2,2,2,2,2,2
Berkersheim,2,2,2,2,2,2
Bockenheim,22,22,22,22,22,22
Bonames,8,8,8,8,8,8
Bornheim,4,4,4,4,4,4
Dornbusch,6,6,6,6,6,6
Eckenheim,6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [49]:
print('There are {} uniques categories.'.format(len(frankfurt_venues['Venue Category'].unique())))

There are 172 uniques categories.


In [51]:
# one hot encoding
frankfurt_onehot = pd.get_dummies(frankfurt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
frankfurt_onehot['Neighborhood'] = frankfurt_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [frankfurt_onehot.columns[-1]] + list(frankfurt_onehot.columns[:-1])
frankfurt_onehot = frankfurt_onehot[fixed_columns]

frankfurt_onehot.head(25)

,Neighborhood,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,Bakery,Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Currywurst Joint,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Friterie,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Lake,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Post Office,Pub,Radio Station,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Well,Whisky Bar,Wine Bar
0,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [52]:
frankfurt_onehot.shape

(646, 173)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
frankfurt_grouped = frankfurt_onehot.groupby('Neighborhood').mean().reset_index()
print(frankfurt_grouped.shape)
frankfurt_grouped

(44, 173)


,Neighborhood,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Apple Wine Pub,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,Bakery,Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Currywurst Joint,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food & Drink Shop,Food Court,French Restaurant,Friterie,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Lake,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Museum,Music Store,Music Venue,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Post Office,Pub,Radio Station,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shopping Mall,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Well,Whisky Bar,Wine Bar
0,Altstadt,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.200000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.2,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.2,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000
1,Bahnhofsviertel,0.01,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.030000,0.000000,0.00,0.00,0.030000,0.050000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.030,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.030000,0.000000,

### London Data Analysis

In [3]:
# Get wikipedia dataset for Frankfurt
# https://en.wikipedia.org/wiki/Frankfurt
London_html = wp.page("List_of_areas_of_London").html().encode("UTF-16")

# Get the table containing the 46 city districts of Frankfurt on the Wikipedia page
df_London = pd.read_html(London_html, header = 0)[1][['Location']] 
print(df_London.shape)
df_London.head()

(533, 1)


,Location
0,Abbey Wood
1,Acton
2,Addington
3,Addiscombe
4,Albany Park
